<a href="https://colab.research.google.com/github/shaktimann/COVID-ImageClassification-DeepLearning-Models_Ensemble/blob/master/mcts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Game Playing Agents #

In this project, we will implement a variation of **Minimax** and **Monte Carlo Tree Search** to play a simple strategic game.

The game is played on a 2D grid on which there are a number of enemy bases. The agent must destroy the enemy bases to win the game. Here is an example of a 5x5 map:``````

```
[] T3 [] B1 [] 
[] [] T2 [] [] 
A  [] [] [] [] 
[] T1 [] T0 [] 
[] [] T4 [] B0
```
The agent is specified by `A`. There are two bases, `B0` and `B1`. There are five towers: `T0` through `T4`.

Towers are and bases are automatically destroyed when the agent encounters them and they are no longer invincible. But when the agent encounters invincible towers and bases, the agent is automatically killed. This game emulates the high-level strategic planning, wherein an encounter between the agent and a tower or base might represent a drawn-out firefight.

However, the towers must be destroyed in a specific order. The bases are ``guarded`` by towers. This means bases are invulnerable until the guarding towers are destroyed. Some towers may also be ``guarded`` by other towers and thus be invulnerable until the guarding towers are destroyed. 

For example, in the above map it might be the case that:
```
B0: guarded by T0
B1: guarded by T1
T0: guarded by T3 and T4
T1: guarded by T2
```
The game can be won by destroying towers in the following order: T2, T1, B1, T3, T4, T0, B0.
There are many possible orderings and the agent must discover one of the correct orderings.

The guard configuration is represented internally in the game engine with a dictionary that you will be able to see but the agent will not be able to see:
```
{'B0': ['T0'], 'B1': ['T1'], 'T1': ['T2'], 'T0': ['T3', 'T4']}
```
The agent must learn one of the correct orderings through trial and error using either a **Minimax**-like algorithm or through **Monte Carlo Tree Search**.

To facilitate this, we provide a means of scoring the agent's attempts. The agent gets -100 points for dying, either from encountering a tower or base that is still guarded, or by going off the edge of the map. 
The agent gets +10 points for each tower destroyed.
The agent gets +50 points for each based destroyed.
The game ends when the agent is killed or when all bases are destroyed.

The project comes is a set number of hand-crafted maps, but is also capable of generating new maps of arbitrary complexity.





# Install and import

In [ ]:
!pip install graphviz
!pip install pydot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from typing import NamedTuple
from copy import copy, deepcopy
from enum import Enum
from functools import partial
import random
import math
import graphviz
import pydot
from IPython.display import Image

# Data Structures

In [ ]:
### Entity class: can be agents, towers, or bases
class Entity():
  def __init__(self, name, x, y, status):
    self.name = name          # Name
    self.x = x                # x position
    self.y = y                # y position
    self.status = status      # status from Status ENUM

### State object
class State():
  def __init__(self, agent, towers = [], bases = [], terminal = False, rows=3, cols=3, guards = {}):
    self.agent = agent            # pointer to an agent Entity
    self.towers = towers          # list of tower Entity objects
    self.bases = bases            # list of base Entity objects
    self.terminal = terminal      # Is the state terminal (boolean)
    self.rows = rows              # Number of rows in grid
    self.cols = cols              # number of columns in grid
    self.guards = guards          # A dictionary of entity names (string) where 
                                  # values are lists of entity names
                                  # E.g. {B0: [T1, T2]} means base B0 is guarded by towers T1 and T2
                                  # and B0 will only be vulnerable when T1 and T2 are Status.DEAD.

#Agent Actions

These functions describe the actions the agent can perform.

In [ ]:
def agent_east(state):
  state.agent.x = state.agent.x + 1
  return state

def agent_north(state):
  state.agent.y = state.agent.y - 1
  return state

def agent_south(state):
  state.agent.y = state.agent.y + 1
  return state

def agent_west(state):
  state.agent.x = state.agent.x - 1
  return state

# Enumerated Types

In [ ]:
### Types of entities
class EntityTypes(Enum):
  AGENT = 'A'
  BASE = 'B'
  TOWER = 'T'

### Agents, towers, and bases can be alive or dead
class Status(Enum):
  ALIVE = 1
  DEAD = 0

### List of available actions
### Values are functions. 
### e.g., Call Actions.EAST.value(state)
class Actions(Enum):
  EAST = partial(agent_east)
  SOUTH = partial(agent_south)
  NORTH = partial(agent_north)
  WEST = partial(agent_west)

# Drawing the Game State

In [ ]:
### Make a grid as a list of lists

def make_grid(cols, rows):
  grid = [[0 for col in range(cols)] for row in range(rows)]
  return grid

### Put names of agents, towers, and bases on a grid (when alive)

def draw_state(state):
  grid = make_grid(state.cols, state.rows)
  for tower in state.towers:
    if tower.status == Status.ALIVE and tower.x >= 0 and tower.x < state.cols and tower.y >=0 and tower.y < state.rows:
      grid[tower.y][tower.x] = tower.name
  for base in state.bases:  
    if base.status == Status.ALIVE and base.x >= 0 and base.x < state.cols and base.y >=0 and base.y < state.rows:
      grid[base.y][base.x] = base.name
  if state.agent.x >= 0 and state.agent.x < state.cols and state.agent.y >=0 and state.agent.y < state.rows:
    grid[state.agent.y][state.agent.x] = EntityTypes.AGENT.value
  return grid

### Make game state easy to view
### indent: shift everything to the right by a number of spaces

def pretty_print(state, indent = 0):
  # Make a grid with agent, towers, and bases
  grid = draw_state(state)
  # Now redraw the grid line by line
  for j in range(len(grid)):
    line = ''
    for i in range(len(grid[j])):
      c = grid[j][i]
      if c == 0:
        c = '[]'
      if len(c) < 2:
        c = c + " "
      line = line + c + " "  
    print(' '*indent + line)
  print(' '*indent + "  guards", state.guards)
  print(' '*indent + "  agent", state.agent.status.name)
  print(' '*indent + "  towers", [tower.name + ": " + str(tower.status.name) for tower in state.towers])
  print(' '*indent + "  bases", [base.name + ": " + str(base.status.name) for base in state.bases])
  print(' '*indent + "  terminal", state.terminal)

# Make a Random Game

In [ ]:
### Make the towers, bases, and their dependencies (who defends who).
### Each base is always defended by at least one tower
### num_towers must be equal to or greater than num_bases

def make_towers_and_bases(num_bases = 1, num_towers = 1):
  # Make bases
  bases = [str(EntityTypes.BASE.value)+str(b) for b in list(range(num_bases))]
  # Make towers
  towers = [str(EntityTypes.TOWER.value)+str(b) for b in list(range(num_towers))]
  # Initialize defended hash
  guards = {}
  # initially each base should have a tower
  for i, base in enumerate(bases):
    guards[base] = [towers[i]]
  # Keep track of used bases and towers. All bases are used up and some towers equal to number of bases
  used = bases + towers[:i+1]
  # Keep track of remaining towers
  remaining_towers = towers[i+1:]
  # Iterate through remaining towers, assigning them to defend
  while len(remaining_towers) > 0:
    next_tower = remaining_towers.pop(0)
    pick = used[random.randrange(0, len(used))]
    if pick not in guards:
      guards[pick] = []
    guards[pick].append(next_tower)
    used.append(pick)
  return bases, towers, guards

In [ ]:
### Make towers and bases then assign them to grid locations
### Make an initial state object

def make_random_game(num_bases = 1, num_towers = 1):
  # Make bases and towers and assign defenders
  bases, towers, guards = make_towers_and_bases(num_bases, num_towers)
  # Everything altogether
  all = bases + towers
  # We will always have 5 rows
  # Compute how many columns we need
  num_cols = int(math.ceil(len(all)/4.0)) + 3
  # Make a set of valid positions
  positions = []
  # Iterate through all the rows and columsn
  # j loop is for rows 
  for j in range(4):
    adjust = 0 # Sometimes we need fewer slots on odd rows
    if num_cols % 2 == 0:
      adjust = 1
    #i loop is for columns
    for i in range(int(num_cols/2) - ((j%2) * adjust)):
      # A column position
      col = (i * 2) + 1
      # If we are on odd rows, add an offset
      if (j % 2) == 1:
        col = col + 1
      # Compute the row number
      row = j
      # Skip the middle row
      if row > 1:
        row = row + 1
      # Store the valid position
      positions.append((col, row))
  # For storing entities when we make them
  tower_entities = []
  base_entities = []
  # Iterate through towers and make entities
  for tower in towers:
    # Grab a valid position
    pos = positions.pop(random.randrange(0, len(positions)))
    # Make a tower entity
    tower_entities.append(Entity(name = tower, x = pos[0], y = pos[1], status = Status.ALIVE))
  # Iterate through bases and make entities
  for base in bases:
    # Grab a valid position
    pos = positions.pop(random.randrange(0, len(positions)))
    # Make a base entity
    base_entities.append(Entity(name = base, x = pos[0], y = pos[1], status = Status.ALIVE))
  # Build the state object
  state = State(agent = Entity(name = "agent", x = 0, y = 2, status = Status.ALIVE),
                towers = [tower for tower in tower_entities],
                bases = [base for base in base_entities],
                terminal = False,
                rows = 5,
                cols = num_cols,
                guards = guards)
  return state

In [ ]:
random_game_state = make_random_game(2, 3)
pretty_print(random_game_state)

[] [] [] T2 [] 
[] [] B1 [] [] 
A  [] [] [] [] 
[] T1 [] [] [] 
[] [] T0 [] B0 
  guards {'B0': ['T0', 'T2'], 'B1': ['T1']}
  agent ALIVE
  towers ['T0: ALIVE', 'T1: ALIVE', 'T2: ALIVE']
  bases ['B0: ALIVE', 'B1: ALIVE']
  terminal False


# Hard-coded initial states

In [ ]:
state5x3 = State(agent=Entity(name="agent",x=0,y=1,status=Status.ALIVE), 
                 towers=[Entity(name="t0", x=2, y=0,status=Status.ALIVE)], 
                 bases=[Entity(name="b0", x=4, y=2,status=Status.ALIVE)], 
                 terminal = False,
                 rows = 3,
                 cols = 5,
                 guards = {'b0': ['t0']})

state3x3 = State(agent=Entity(name="agent",x=0,y=1,status=Status.ALIVE), 
                 towers=[Entity(name="t0",x=1, y=0,status=Status.ALIVE)], 
                 bases=[Entity(name="b0",x=2, y=2,status=Status.ALIVE)], 
                 terminal = False,
                 rows = 3,
                 cols = 3,
                 guards = {'b0': ['t0']})

state3x3_t = State(agent=Entity(name="agent",x=1,y=1,status=Status.ALIVE), 
                 towers=[Entity(name="t0",x=1, y=0,status=Status.ALIVE),
                         Entity(name="t1",x=1, y=2,status=Status.ALIVE),
                         Entity(name="t2",x=0, y=1,status=Status.ALIVE)], 
                 bases=[Entity(name="b0",x=2, y=1,status=Status.ALIVE)], 
                 terminal = False,
                 rows = 3,
                 cols = 3,
                 guards = {'t2': ['t1'], 't1':['t0'], 'b0':['t2']})

state2towers = State(agent=Entity(name="agent",x=0,y=1,status=Status.ALIVE), 
                     towers=[Entity(name="t0",x=1,y=0,status=Status.ALIVE),
                             Entity(name="t1",x=1,y=2,status=Status.ALIVE)], 
                     bases=[Entity(name="b0",x=2, y=1,status=Status.ALIVE)], 
                     terminal = False,
                     rows = 3,
                     cols = 3,
                     guards = {'b0': ['t0', 't1']})

state_big = State(agent=Entity(name="agent", x=0, y=1, status = Status.ALIVE),
                  towers = [Entity(name="T0",x=3,y=0,status=Status.ALIVE),
                            Entity(name="T1",x=2,y=2,status=Status.ALIVE),
                            Entity(name="T2",x=2,y=0,status=Status.ALIVE),
                            Entity(name="T3",x=3,y=2,status=Status.ALIVE)],
                  bases = [Entity(name="B0",x=4, y=0,status=Status.ALIVE),
                           Entity(name="B1",x=1, y=2,status=Status.ALIVE)],
                  terminal = False,
                  rows = 3,
                  cols = 5,
                  guards = {'B0': ['T0'], 'B1': ['T1'], 'T0': ['T2'], 'T1': ['T3']})

state_big2 = State(agent=Entity(name="agent", x=0, y=0, status = Status.ALIVE),
                   towers = [Entity(name="T0",x=5,y=9,status=Status.ALIVE),
                             Entity(name="T1",x=9,y=9,status=Status.ALIVE),
                             ],
                   bases = [Entity(name="B0",x=0, y=5,status=Status.ALIVE),
                            ],
                   terminal = False,
                   rows = 10,
                   cols = 10,
                   guards = {'B0': ['T0'], 'T0': ['T1']})

big_test_map = State(agent=Entity(name="agent", x=0, y=2, status = Status.ALIVE),
                      towers = [Entity(name="T3",x=1,y=0,status=Status.ALIVE),
                                Entity(name="T2",x=2,y=1,status=Status.ALIVE),
                                Entity(name="T1",x=1,y=3,status=Status.ALIVE),
                                Entity(name="T0",x=3,y=3,status=Status.ALIVE),
                                Entity(name="T4",x=2,y=4,status=Status.ALIVE)
                                ],
                  bases = [Entity(name="B0",x=4, y=4,status=Status.ALIVE),
                           Entity(name="B1",x=3, y=0,status=Status.ALIVE)
                           ],
                  terminal = False,
                  rows = 5,
                  cols = 5,
                  guards = {'B0': ['T0'], 'B1': ['T1'], 'T1': ['T2'], 'T0': ['T3', 'T4']})

# Initialize game state

In [277]:
state_to_test = state2towers
init_state = deepcopy(state_to_test)
pretty_print(init_state)

[] t0 [] 
A  [] b0 
[] t1 [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: ALIVE', 't1: ALIVE']
  bases ['b0: ALIVE']
  terminal False


# Compute the game score

In [ ]:
### +50 for each dead base
### +10 for each dead tower
### -100 for dead agent
### -1 for each depth (length of solution)

def score(state, depth = 0):
  score = 0
  # Is the agent dead?
  if state.agent.status == Status.DEAD:
    score = score - 100
#  else:
  # Any dead towers?
  for tower in state.towers:
    if tower.status == Status.DEAD:
      score = score + 100
  # Any dead bases?
  for base in state.bases:
    if base.status == Status.DEAD:
      score = score + 200
# Length penalty
  score = score - depth
  return score

# Executing Actions

This section contains the logic for the rules of the game--what happens when different actions are executed under different circumstances causing the agent or towers or bases to die. The `do_action()` is the core function of the game.

Helpers:

In [ ]:
### Figure out if an entity (string name of a base or tower) is alive or not

def is_alive(entity, state):
  # Check to see if entity is the agent
  if state.agent.name == entity:
    return state.agent.name.status == Status.ALIVE
  # Check to see if entity is a tower
  for tower in state.towers:
    if tower.name == entity:
      return tower.status == Status.ALIVE
  # Check to see if entity is a base
  for base in state.bases:
    if base.name == entity:
      return base.status == Status.ALIVE
  return False

### Figure out if a tower or base (name string) is defended.
### An entity is not defended if every defending entity is dead.

def guarded(entity, state):
  if entity in state.guards:
    for guard in state.guards[entity]:
      if is_alive(guard, state):
        return True
  return False  

The `do_action()` function

In [ ]:
### Execute an action
### Action is from Actions ENUM.

def do_action(action, state):
  # Make a deep copy of the state
  new_state = deepcopy(state)
  # Call the action function
  new_state = action.value(new_state)
  # Grab a reference to the agent for convenience
  agent = new_state.agent
  # If agent has gone off the board, then it is dead
  if agent.x < 0 or agent.y < 0 or agent.x >= state.cols or agent.y >= state.rows:
    agent.status = Status.DEAD
    new_state.terminal = True
    #new_state = deepcopy(state) # don't die when going off the edge, just don't move
  else:
    # Did the agent encounter a tower?
    for tower in new_state.towers:
      if tower.status == Status.ALIVE and agent.x == tower.x and agent.y == tower.y:
        # Is the tower defended?
        if guarded(tower.name, state):
          # Tower is defended so agent is dead
          agent.status = Status.DEAD
          new_state.terminal = True
        else:
          # Tower was not defended so tower is dead
          tower.status = Status.DEAD
    # Did the agent encounter a base?
    for base in new_state.bases:
      if base.status == Status.ALIVE and agent.x == base.x and agent.y == base.y:
        # Is the base defended?
        if guarded(base.name, state):
          # Base is defended so agent is dead
          agent.status = Status.DEAD
          new_state.terminal = True
        else:
          # Base was not defended so base is dead
          base.status = Status.DEAD
    # Check for alive bases
    alive_bases = 0
    for base in new_state.bases:
      if base.status == Status.ALIVE:
        alive_bases = alive_bases + 1
    # If there are no alive bases, then the game is over
    if alive_bases == 0:
      new_state.terminal = True
  return new_state

Utility function for driving the agent manually

In [ ]:
### Utility for driving the agent manually.
### Route is a list of actions from Actions ENUM (e.g., [Actions.EAST, Actions.NORTH, etc.])
def test_run(state, route):
  # Print the game state  
  pretty_print(state)
  # Execute each action in route
  for action in route:
    print("\nACTION", action.name, '\n')
    state = do_action(action, state)
    # Print the game state
    pretty_print(state)
  # Print the score
  print("Score:", score(state))
  return state

In [ ]:
dummy_state = test_run(init_state, [Actions.EAST, Actions.NORTH, Actions.EAST])

[] t0 [] 
A  [] b0 
[] t1 [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: ALIVE', 't1: ALIVE']
  bases ['b0: ALIVE']
  terminal False

ACTION EAST 

[] t0 [] 
[] A  b0 
[] t1 [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: ALIVE', 't1: ALIVE']
  bases ['b0: ALIVE']
  terminal False

ACTION NORTH 

[] A  [] 
[] [] b0 
[] t1 [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: DEAD', 't1: ALIVE']
  bases ['b0: ALIVE']
  terminal False

ACTION EAST 

[] [] A  
[] [] b0 
[] t1 [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: DEAD', 't1: ALIVE']
  bases ['b0: ALIVE']
  terminal False
Score: 100


# Maximax

For the student to complete

In [ ]:
### Maximax function
### state is initial state
### actions is the set of possible actions
### max_depth is how deep the search can go before backtracking

def maximax(state, actions = [act for act in Actions], max_depth = float("inf")):
  best_action = None # Return the best action
  best_action_value = -float("inf") # Return the value of the best action (dummy default is -infinity)
  ### YOUR CODE GOES BELOW

  for action in actions:
      new_state = do_action(action, state)
      value, _ = recurse(new_state, actions, 1, max_depth)
      if value > best_action_value:
          best_action_value = value
          best_action = action

  ### YOUR CODE GOES ABOVE
  return best_action, best_action_value

def recurse(state, actions, depth, max_depth):
    if depth == max_depth or state.terminal:
        return score(state, depth), None
    best_action = None
    best_value = -float("inf")
    for action in actions:
        new_state = do_action(action, state)
        value, _ = recurse(new_state, actions, depth+1, max_depth)
        if value > best_value:
            best_value = value
            best_action = action
    return best_value, best_action


Run the agent using the maximax decision-making algorithm. This runs maximax after every action as if the tree is unreliable after execution. In reality it is not, and it is possible to implement some sort of state memorization.

In [ ]:
def run_maximax(state, actions = [act for act in Actions], max_depth = float("inf")):
  # Print the state
  pretty_print(state)
  value = 0
  # Run until terminal state reached
  while not state.terminal:
    # Compute best action
    action, value = maximax(state, actions, max_depth = max_depth)
    print("\nACTION", action.name, "value", value, '\n')
    # Execute the action
    state = do_action(action, state)
    # print the state
    pretty_print(state)

  return value


Test Maximax

In [ ]:
pretty_print(init_state)
action, value = maximax(init_state, max_depth=10)
print("action:", action.name)
print("value:", value)

[] t0 [] 
A  [] b0 
[] t1 [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: ALIVE', 't1: ALIVE']
  bases ['b0: ALIVE']
  terminal False
action: EAST
value: 394


In [ ]:
run_maximax(init_state, max_depth=10)

[] t0 [] 
A  [] b0 
[] t1 [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: ALIVE', 't1: ALIVE']
  bases ['b0: ALIVE']
  terminal False

ACTION EAST value 394 

[] t0 [] 
[] A  b0 
[] t1 [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: ALIVE', 't1: ALIVE']
  bases ['b0: ALIVE']
  terminal False

ACTION SOUTH value 395 

[] t0 [] 
[] [] b0 
[] A  [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: ALIVE', 't1: DEAD']
  bases ['b0: ALIVE']
  terminal False

ACTION NORTH value 396 

[] t0 [] 
[] A  b0 
[] [] [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: ALIVE', 't1: DEAD']
  bases ['b0: ALIVE']
  terminal False

ACTION NORTH value 397 

[] A  [] 
[] [] b0 
[] [] [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: DEAD', 't1: DEAD']
  bases ['b0: ALIVE']
  terminal False

ACTION EAST value 398 

[] [] A  
[] [] b0 
[] [] [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: DEAD', 't1: DEAD']
  bases ['b0: ALIVE']
 

399

# Monte Carlo Tree Search (MCTS)

Tree node data structure

In [ ]:
class Node():
  def __init__(self, state, Q = 0, action = None, parent = None, depth = 0):
    self.state = state             # State of the game
    self.Q = Q                     # backpropagaged reward, summed across visits
    self.count = 0                 # number of times visited
    self.parent = parent           # Pointer to parent node
    self.children = []             # Pointer to children nodes
    self.untried_children = []     # Pointer to children that haven't been visited [optional]
    self.action = action           # Action that generated this state
    self.depth = depth             # Depth in tree

Helper to render an image of the game tree using graphviz. Find the file in the file system. you may want to use this to inspect the tree your MCTS implementation generates (it will return the root of the tree). Alternatively, it can be very instructive to generate a series of images after every loop of the MCTS algorithm, to see if the tree is being expanded correctly. This slows things down substantially, however.

In [ ]:
### Render an image to file.
### root: the root node of the search tree
### filename: the name of the file (without filetype suffic)
### c: the constant for upper confidence bound calculation

def render_mcts_tree(root, filename, c = 1):
  # Create graphviz root container
  dot = graphviz.Digraph()
  # Queue for exhaustive search through tree
  queue = [root]
  # Iterate while there are remaining nodes
  while len(queue) > 0:
    # Get current node
    node = queue.pop(0)
    # Node label is the count, Q, then both parts of UCB
    label = "count:" + str(node.count) + " Q:" + str(node.Q) + ' exp:'+ str((node.Q/node.count) if node.count > 0 else "n/a")
    # Create the node, with unique identifier for node. Terminal states will be red.
    dot.node(name=str(id(node)), label=str(label), color="red" if node.state.terminal else "black")
    # If there is a parent, create an edge, label it with action name
    if node.parent is not None:
      dot.edge(str(id(node.parent)), str(id(node)), label=str(node.action.name))
    # Grab children and add to queue
    for child in node.children:
      queue.append(child)
  # Render to file
  dot.render(filename + '.gv', view=False)
  (graph,) = pydot.graph_from_dot_file(filename + '.gv')
  graph.write_png(filename + '.png') 

Upper confidence bound calculation. `c` is the constant, which defaults to 1.



In [294]:
### The upper confidence bound for the node
### c: the constant applied to control exploration.

def ucb(node, c = 1):
  result = None
  ### YOUR CODE BELOW HERE
  if node.count==0:
    return float("inf")
  if node.parent is None:
      return node.Q/node.count
  result = node.Q/node.count + c * math.sqrt(math.log(node.parent.count)/node.count)
  ### YOUR CODE ABOVE HERE
  return result

MCTS function implementation goes here.

In [298]:
def select(node, c):
  max_uct = float("-inf")
  best_child = None
  for child in node.children:
      uct = ucb(child, c)
      if uct > max_uct:
        max_uct = uct
        best_child = child
  return best_child

def is_valid(state, action):
  if action == Actions.NORTH and state.agent.x-1>=0:
    return True
  if action == Actions.SOUTH and state.agent.x+1<=state.rows:
    return True
  if action == Actions.WEST and state.agent.y-1>=0:
    return True
  if action == Actions.EAST and state.agent.y+1<=state.cols:
    return True
  return False

def expand(node, actions):
  for action in actions:
    if (is_valid(node.state, action)):
      child_state = do_action(action, node.state)
      child = Node(child_state, action=action, parent=node, depth=node.depth+1)
      node.children.append(child)
  return random.choice(node.children)

def simulate(node, actions, num_rollouts, max_depth):
  state = node.state
  for i in range(max_depth):
    if state.terminal:
      return node.Q
    action = random.choice(actions)
    state = do_action(action, state)
  return node.Q

def rollout(node, actions, num_rollouts, max_rollout_depth):
    result_sum = 0
    for i in range(num_rollouts):
        state = node.state
        depth = node.depth
        while not state.terminal and depth < max_rollout_depth:
            action = random.choice(actions)
            state = do_action(action, state)
            depth += 1
        result_sum += score(state, depth)
    result = result_sum
    return result

def backpropagate(node, reward):
    while node is not None:
      node.count += 1
      node.Q += reward
      node = node.parent

### MCTS 
### State is the initial state
### actions is the list of available actions from Actions ENUM
### num_iters: number of times to expand the tree
### num_rollouts: number of times to try rollouts per visit
### c: constant for UCB calculation
def mcts(state, actions = [act for act in Actions], num_iters = 10, num_rollouts = 10, max_rollout_depth = 20, c = 1):
  # Make root  node
  root = Node(state=state)
  best_action = None
  working_depth = 0
  working_dist = 0
  working = root
  ### YOUR CODE BELOW HERE

  for action in actions:
    if(is_valid(root.state, action)):
      child_state = do_action(action, root.state)
      child = Node(child_state, action=action, parent=root, depth=root.depth+1)
      root.children.append(child)

  for i in range(num_iters):
      node = root   

      # Selection phase
      while len(node.children) != 0:
        node = select(node, c)

      # Expansion phase
      if(node.count!=0):
        temp = expand(node, actions)
        if(temp is not None): 
          node = temp

      # Simulation phase
      reward = rollout(node, actions, num_rollouts, max_rollout_depth)

      # Backpropagation phase
      backpropagate(node, reward)

  best_child = max(root.children, key=lambda child: child.Q)
  best_action = best_child.action
  ### YOUR CODE ABOVE HERE
  return best_action, root


Debugging hints: 
* Render the tree after every iteration with a different name so you can see how the tree changes after every iteration.
* Print out the `root.Q/root.count` after every iteration. This number should go down over time.
* Print out the depth and agent's coordinates of the node that is expanded in every iteration to see what states the search is focusing on. The game is simple enough that where the agent is in the grid is indicative of progress.
* Print out the scores your rollouts achieve (if you are not doing a lot of rollouts).
* Experiment with different numbers of iterations, number of rollouts, max rollout depths, and `c` values.
* Make sure depth is being handled correctly in expansions and in rollouts.

Test MCTS

In [299]:
action, tree = mcts(init_state, num_iters=500, num_rollouts=10, max_rollout_depth=2, c=0)
print(action)

Actions.EAST


Render the tree as an image

In [ ]:
render_mcts_tree(tree, filename = "tree")

Helper to get a node in the full tree. Route is a list of Actions that the agent would have taken.



In [300]:
### Get a node from full tree
### route: list of actions from Actions enum (e.g., [Actions.EAST, Actions.NORTH, etc.])

def get_node_by_route(root, route):
  # Start at the route
  node = root
  # Follow the route
  for act in route:
    # Find the child that the action should lead to
    for child in node.children:
      if child.action == act:
        node = child
        break
  return node

In [301]:
example_node = get_node_by_route(tree, [Actions.EAST, Actions.NORTH])
pretty_print(example_node.state)
print("score=",score(example_node.state))
print("q=", example_node.Q, "count=", example_node.count)

[] A  [] 
[] [] b0 
[] t1 [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: DEAD', 't1: ALIVE']
  bases ['b0: ALIVE']
  terminal False
score= 100
q= 2810 count= 6


Run the agent with the MCTS algorithm making decisions at every step. The tree must be regenerated every time because sampling will mean different information is available. Depending on how the `num_iters` and `num_rollouts` is set, the agent may be more or less successful. Also, the agent may take different paths every time.

In [302]:
### Run the MCTS agent
### state: init state
### actions: list of actions from Actions enum
### num_iters: number of times to expand the tree
### num_rollouts: number of rollouts per visit
### c: constant for ucb calculation

def run_mcts(state, actions = [act for act in Actions], num_iters = 10, num_rollouts = 10, c = 1, max_rollout_depth = 100):
  # Print the start
  pretty_print(state)
  # Keep track of actions
  executed_actions = []
  # Run until a terminal state is reached
  while not state.terminal:
    # Compute the best action
    action, _ = mcts(state=state, actions=actions, num_iters=num_iters, num_rollouts=num_rollouts, c=c, max_rollout_depth=max_rollout_depth)
    print("\nACTION", action.name, '\n')
    # Append action to the list of stored actions
    executed_actions.append(action)
    # Execute the action
    state = do_action(action, state)
    # Print the new state
    pretty_print(state)
    print("\nactions:", [a.name for a in executed_actions])
    print("\nscore=", score(state, depth=len(executed_actions)))
  return state, executed_actions


In [303]:
end_state, actions = run_mcts(init_state, num_iters=500, num_rollouts=10, max_rollout_depth=10, c=1)

[] t0 [] 
A  [] b0 
[] t1 [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: ALIVE', 't1: ALIVE']
  bases ['b0: ALIVE']
  terminal False

ACTION EAST 

[] t0 [] 
[] A  b0 
[] t1 [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: ALIVE', 't1: ALIVE']
  bases ['b0: ALIVE']
  terminal False

actions: ['EAST']

score= -1

ACTION SOUTH 

[] t0 [] 
[] [] b0 
[] A  [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: ALIVE', 't1: DEAD']
  bases ['b0: ALIVE']
  terminal False

actions: ['EAST', 'SOUTH']

score= 98

ACTION NORTH 

[] t0 [] 
[] A  b0 
[] [] [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: ALIVE', 't1: DEAD']
  bases ['b0: ALIVE']
  terminal False

actions: ['EAST', 'SOUTH', 'NORTH']

score= 97

ACTION NORTH 

[] A  [] 
[] [] b0 
[] [] [] 
  guards {'b0': ['t0', 't1']}
  agent ALIVE
  towers ['t0: DEAD', 't1: DEAD']
  bases ['b0: ALIVE']
  terminal False

actions: ['EAST', 'SOUTH', 'NORTH', 'NORTH']

score= 196

ACTION SOUTH 

[] []

### Running Maximax and MCTS on Larger Map


In [304]:
state_to_test = big_test_map
init_state = deepcopy(state_to_test)
pretty_print(init_state)

[] T3 [] B1 [] 
[] [] T2 [] [] 
A  [] [] [] [] 
[] T1 [] T0 [] 
[] [] T4 [] B0 
  guards {'B0': ['T0'], 'B1': ['T1'], 'T1': ['T2'], 'T0': ['T3', 'T4']}
  agent ALIVE
  towers ['T3: ALIVE', 'T2: ALIVE', 'T1: ALIVE', 'T0: ALIVE', 'T4: ALIVE']
  bases ['B0: ALIVE', 'B1: ALIVE']
  terminal False


Run maximax on the big map

In [305]:
maximax_val = run_maximax(init_state, max_depth=10)
print("Maximax Value: ", maximax_val)

[] T3 [] B1 [] 
[] [] T2 [] [] 
A  [] [] [] [] 
[] T1 [] T0 [] 
[] [] T4 [] B0 
  guards {'B0': ['T0'], 'B1': ['T1'], 'T1': ['T2'], 'T0': ['T3', 'T4']}
  agent ALIVE
  towers ['T3: ALIVE', 'T2: ALIVE', 'T1: ALIVE', 'T0: ALIVE', 'T4: ALIVE']
  bases ['B0: ALIVE', 'B1: ALIVE']
  terminal False

ACTION EAST value 390 

[] T3 [] B1 [] 
[] [] T2 [] [] 
[] A  [] [] [] 
[] T1 [] T0 [] 
[] [] T4 [] B0 
  guards {'B0': ['T0'], 'B1': ['T1'], 'T1': ['T2'], 'T0': ['T3', 'T4']}
  agent ALIVE
  towers ['T3: ALIVE', 'T2: ALIVE', 'T1: ALIVE', 'T0: ALIVE', 'T4: ALIVE']
  bases ['B0: ALIVE', 'B1: ALIVE']
  terminal False

ACTION EAST value 490 

[] T3 [] B1 [] 
[] [] T2 [] [] 
[] [] A  [] [] 
[] T1 [] T0 [] 
[] [] T4 [] B0 
  guards {'B0': ['T0'], 'B1': ['T1'], 'T1': ['T2'], 'T0': ['T3', 'T4']}
  agent ALIVE
  towers ['T3: ALIVE', 'T2: ALIVE', 'T1: ALIVE', 'T0: ALIVE', 'T4: ALIVE']
  bases ['B0: ALIVE', 'B1: ALIVE']
  terminal False

ACTION NORTH value 490 

[] T3 [] B1 [] 
[] [] A  [] [] 
[] [] [] [] [

Run MCTS on big map

In [306]:
end_state, actions = run_mcts(init_state, num_iters=500, num_rollouts=10, max_rollout_depth=10, c=1)
print("MCTS Value: ", score(end_state, depth=len(actions)))

[] T3 [] B1 [] 
[] [] T2 [] [] 
A  [] [] [] [] 
[] T1 [] T0 [] 
[] [] T4 [] B0 
  guards {'B0': ['T0'], 'B1': ['T1'], 'T1': ['T2'], 'T0': ['T3', 'T4']}
  agent ALIVE
  towers ['T3: ALIVE', 'T2: ALIVE', 'T1: ALIVE', 'T0: ALIVE', 'T4: ALIVE']
  bases ['B0: ALIVE', 'B1: ALIVE']
  terminal False

ACTION EAST 

[] T3 [] B1 [] 
[] [] T2 [] [] 
[] A  [] [] [] 
[] T1 [] T0 [] 
[] [] T4 [] B0 
  guards {'B0': ['T0'], 'B1': ['T1'], 'T1': ['T2'], 'T0': ['T3', 'T4']}
  agent ALIVE
  towers ['T3: ALIVE', 'T2: ALIVE', 'T1: ALIVE', 'T0: ALIVE', 'T4: ALIVE']
  bases ['B0: ALIVE', 'B1: ALIVE']
  terminal False

actions: ['EAST']

score= -1

ACTION NORTH 

[] T3 [] B1 [] 
[] A  T2 [] [] 
[] [] [] [] [] 
[] T1 [] T0 [] 
[] [] T4 [] B0 
  guards {'B0': ['T0'], 'B1': ['T1'], 'T1': ['T2'], 'T0': ['T3', 'T4']}
  agent ALIVE
  towers ['T3: ALIVE', 'T2: ALIVE', 'T1: ALIVE', 'T0: ALIVE', 'T4: ALIVE']
  bases ['B0: ALIVE', 'B1: ALIVE']
  terminal False

actions: ['EAST', 'NORTH']

score= -2

ACTION EAST 

[] T3 

# Grading

For Maximax and MCTS each, you will receive points based on how well your agent performs on ``big_test_map`` map:
- 5: > 800
- 4: > 600
- 3: > 400
- 2: > 190
- 1: > 97

Total: 10 points

For Maximax and MCTS each, you will receive points based on how well your agents performs on ``state2towers`` (state2towers):
- 5: > 340
- 4: > 240
- 3: > 140
- 2: > 90
- 1: > 40

Total: 10 points

For ``ucb()`` implementation you will be tested on the number of unit-tests passed. Max score will be 5 points.

### UCB Testcases

In [295]:
import unittest

In [296]:
class TestUCB(unittest.TestCase):
  noParent = Node(None, Q=10, action=None, parent=None, depth=0)
  noParent.count = 2

  child = Node(None, Q=7, action=None, parent=noParent, depth=0)
  child.count = 2

  sec_child = Node(None, Q=12, action=None, parent=noParent, depth=0)
  sec_child.count=3

  rand1 = Node(None, Q=284, action=None, parent=None, depth=0)
  rand1.count = 27

  rand2 = Node(None, Q=742, action=None, parent=rand1, depth=0)
  rand2.count = 321

  rand3 = Node(None, Q=412, action=None, parent=rand2, depth=0)
  rand3.count = 200

  def test_no_explore(self):
    # self, state, Q = 0, action = None, parent = None, depth = 0
    self.assertAlmostEqual(ucb(self.noParent, 1), 5, places=3)

  def test_no_c(self):
    self.assertAlmostEqual(ucb(self.child), 4.088705011257737, places=3)

  def test_with_c(self):
    self.assertAlmostEqual(ucb(self.sec_child, 5), 6.4033781443348055, places=3)

  def test_rand1_1(self):
    self.assertAlmostEqual(ucb(self.rand1), 10.518518518518519, places=3)

  def test_rand1_2(self):
    self.assertAlmostEqual(ucb(self.rand1, 67), 10.518518518518519, places=3)

  def test_rand2_1(self):
    self.assertAlmostEqual(ucb(self.rand2), 2.4128546819799297, places=3)

  def test_rand2_2(self):
    self.assertAlmostEqual(ucb(self.rand2, 34), 5.756685355541906, places=3)
  
  def test_rand3_1(self):
    self.assertAlmostEqual(ucb(self.rand3, 1), 2.229874087534415, places=3)
  
  def test_rand3_2(self):
    self.assertAlmostEqual(ucb(self.rand3, 15), 4.608111313016225, places=3)
  
  def test_rand3_3(self):
    self.assertAlmostEqual(ucb(self.rand3, 5), 2.9093704376720746, places=3)

####Running Tests for UCB and Scoring

In [297]:
res = unittest.main(argv=[''], verbosity=2, exit=False)
total_ucb_score = ((10 - len(res.result.failures) - len(res.result.errors))/10)*5
print( "Total Points Acheived:", total_ucb_score)

test_no_c (__main__.TestUCB) ... ok
test_no_explore (__main__.TestUCB) ... ok
test_rand1_1 (__main__.TestUCB) ... ok
test_rand1_2 (__main__.TestUCB) ... ok
test_rand2_1 (__main__.TestUCB) ... ok
test_rand2_2 (__main__.TestUCB) ... ok
test_rand3_1 (__main__.TestUCB) ... ok
test_rand3_2 (__main__.TestUCB) ... ok
test_rand3_3 (__main__.TestUCB) ... ok
test_with_c (__main__.TestUCB) ... ok

----------------------------------------------------------------------
Ran 10 tests in 0.023s

OK


Total Points Acheived: 5.0
